In [0]:
import pandas as pd

In [0]:
file_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-TensorFlow-Workshop/master/Chapter06/dataset/connect-4.csv'

In [0]:
data = pd.read_csv(file_url)
data.head()

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6,c1,c2,c3,c4,c5,c6,d1,d2,d3,d4,d5,d6,e1,e2,e3,e4,e5,e6,f1,f2,f3,f4,f5,f6,g1,g2,g3,g4,g5,g6,class
0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,2,1,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,1,2,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,2,1,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,1,2,1,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2
4,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,1,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [0]:
target = data.pop('class')

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [0]:
!pip install keras-tuner
import kerastuner as kt

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout

In [0]:
tf.random.set_seed(8)

In [0]:
def model_builder(hp):
  model = tf.keras.Sequential()

  hp_l2 = hp.Choice('l2', values = [0.1, 0.01, 0.001, 0.0001])

  reg_fc1 = Dense(512, input_shape=(42,), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
  reg_fc2 = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
  reg_fc3 = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
  reg_fc4 = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
  reg_fc5 = Dense(3, activation='softmax')
  
  model.add(reg_fc1)
  model.add(reg_fc2)
  model.add(reg_fc3)
  model.add(reg_fc4)
  model.add(reg_fc5)
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  optimizer = tf.keras.optimizers.Adam(0.001)
  model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])
  
  return model

In [0]:
tuner = kt.RandomSearch(model_builder, objective='val_accuracy', max_trials=10)   

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [0]:
tuner.search(X_train, y_train, validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [0]:
best_hps = tuner.get_best_hyperparameters()[0]

In [0]:
best_l2 = best_hps.get('l2')
best_l2

0.0001

In [0]:
model = tuner.hypermodel.build(best_hps)

In [0]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
1689/1689 [==============================] - 11s 7ms/step - loss: 0.7437 - accuracy: 0.7146 - val_loss: 0.6458 - val_accuracy: 0.7520
Epoch 2/5
1689/1689 [==============================] - 11s 7ms/step - loss: 0.6038 - accuracy: 0.7734 - val_loss: 0.6003 - val_accuracy: 0.7769
Epoch 3/5
1689/1689 [==============================] - 11s 7ms/step - loss: 0.5498 - accuracy: 0.7952 - val_loss: 0.5431 - val_accuracy: 0.7980
Epoch 4/5
1689/1689 [==============================] - 11s 7ms/step - loss: 0.5167 - accuracy: 0.8109 - val_loss: 0.5655 - val_accuracy: 0.7954
Epoch 5/5
1689/1689 [==============================] - 11s 7ms/step - loss: 0.4920 - accuracy: 0.8216 - val_loss: 0.5247 - val_accuracy: 0.8065
